In [45]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np

In [7]:


list_symbols = pd.read_excel('C:/projeto_python/ibrx.xls')['Código'].values
list_symbols



array(['RRRP3', 'ALPA4', 'ABEV3', 'AMER3', 'ASAI3', 'AZUL4', 'B3SA3',
       'BIDI11', 'BPAN4', 'BBSE3', 'BRML3', 'BBDC3', 'BBDC4', 'BRAP4',
       'BBAS3', 'BRKM5', 'BRFS3', 'BPAC11', 'CRFB3', 'CCRO3', 'CMIG4',
       'CESP6', 'CIEL3', 'COGN3', 'CPLE6', 'CSAN3', 'CPFE3', 'CMIN3',
       'CVCB3', 'CYRE3', 'DXCO3', 'ECOR3', 'ELET3', 'ELET6', 'EMBR3',
       'ENBR3', 'ENGI11', 'ENEV3', 'EGIE3', 'EQTL3', 'EZTC3', 'FLRY3',
       'GGBR4', 'GOAU4', 'GOLL4', 'NTCO3', 'SOMA3', 'HAPV3', 'HYPE3',
       'IGTI11', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LIGT3',
       'RENT3', 'LCAM3', 'LWSA3', 'AMAR3', 'LREN3', 'MGLU3', 'MRFG3',
       'CASH3', 'BEEF3', 'MOVI3', 'MRVE3', 'MULT3', 'PCAR3', 'PETR3',
       'PETR4', 'PRIO3', 'PETZ3', 'PSSA3', 'POSI3', 'QUAL3', 'RADL3',
       'RDOR3', 'RAIL3', 'SBSP3', 'SANB11', 'STBP3', 'CSNA3', 'SLCE3',
       'SULA11', 'SUZB3', 'TAEE11', 'VIVT3', 'TIMS3', 'TOTS3', 'UGPA3',
       'USIM5', 'VALE3', 'VIIA3', 'VBBR3', 'WEGE3', 'YDUQ3'], dtype=object)

In [33]:

list_melhores = []
list_relacao_mm = []
list_precos = []
for ativo in list_symbols:
    soma_fundamentos = 0
    dividend_yield = 0
    chamada_api = yf.Ticker(ativo+'.SA')
    hist = chamada_api.history(period='30mo',interval = '1wk')
    hist_precos = hist['Close']
    hist_dividendos = hist['Dividends'].tz_convert(None)
    hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2022,1,1):datetime(2023,12,31)]
    

    dividend_yield = (hist_dividendos_ultimo_ano.sum()/2)/hist_precos[-1] if hist_precos[-1] > 0 else 0
    
    if dividend_yield > 0.03:
        try:
            balanco_anual = chamada_api.balance_sheet
            balanco_trimestral = chamada_api.quarterly_balance_sheet
            dre_anual = chamada_api.financials
            dfc_anual = chamada_api.cashflow
            dre_trimestral = chamada_api.quarterly_financials
            dfc_trimestral = chamada_api.quarterly_cashflow
            
            if dre_anual.keys()[0].year >= 0:
                balanco = balanco_anual
                dre = dre_anual
                dfc = dfc_anual
                variacao_receita = int(dre.loc['Total Revenue'][0] > dre.loc['Total Revenue'][1])
                variacao_ebit = int(dre.loc['Ebit'][0] > dre.loc['Ebit'][1])
                variacao_caixa = int(dfc.loc['Change In Cash'][0] > dfc.loc['Change In Cash'][1])
                variacao_divida = int(balanco.loc['Total Current Liabilities'][0] < balanco.loc['Total Current Liabilities'][1])
                
                soma_fundamentos = variacao_caixa + variacao_ebit + variacao_receita + variacao_divida
                if soma_fundamentos >= 2:
                    print(ativo,variacao_divida,variacao_caixa,variacao_ebit,variacao_receita,dre_anual.keys()[0])
                    list_melhores.append(ativo)
                    list_relacao_mm.append(abs(hist_precos[-1] / np.mean(hist_precos[-40:])-1))
                    list_precos.append(round(hist_precos[-1],2))
            else:
                continue
        except:
            print(ativo)
            continue
    
    




BRKM5 1 0 1 1 2021-12-31 00:00:00
CPFE3 1 0 1 1 2021-12-31 00:00:00
DXCO3 0 0 1 1 2021-12-31 00:00:00
ELET3 1 0 1 1 2021-12-31 00:00:00
ELET6 1 0 1 1 2021-12-31 00:00:00
EGIE3 0 0 1 1 2021-12-31 00:00:00
FLRY3 0 0 1 1 2021-12-31 00:00:00
GGBR4 0 0 1 1 2021-12-31 00:00:00
GOAU4 0 0 1 1 2021-12-31 00:00:00
JBSS3 0 0 1 1 2021-12-31 00:00:00
MRFG3 0 0 1 1 2021-12-31 00:00:00
BEEF3 0 0 1 1 2021-12-31 00:00:00
PCAR3 1 0 0 1 2021-12-31 00:00:00
PETR3 1 0 1 1 2021-12-31 00:00:00
PETR4 1 0 1 1 2021-12-31 00:00:00
PSSA3 0 1 0 1 2021-12-31 00:00:00
USIM5 0 1 1 1 2021-12-31 00:00:00
VALE3 0 0 1 1 2021-12-31 00:00:00


In [46]:

carteira_df = pd.DataFrame({'MM':list_relacao_mm,'Preços':list_precos}, index=list_melhores).sort_values(by = 'MM',ascending=True)

carteira_df


,MM,Preços
ELET6,0.001089,36.22
ELET3,0.002284,36.53
MRFG3,0.022809,20.17
JBSS3,0.033564,35.42
PCAR3,0.034861,24.82
PSSA3,0.088409,20.86
USIM5,0.099267,13.93
GOAU4,0.105661,12.27
BRKM5,0.106027,45.52
EGIE3,0.134156,43.48


In [52]:
lista_drop = ['ELET6','PETR3']

carteira_limpa = carteira_df.drop(lista_drop)


carteira_limpa


,MM,Preços
ELET3,0.002284,36.53
MRFG3,0.022809,20.17
JBSS3,0.033564,35.42
PCAR3,0.034861,24.82
PSSA3,0.088409,20.86
USIM5,0.099267,13.93
GOAU4,0.105661,12.27
BRKM5,0.106027,45.52
EGIE3,0.134156,43.48
GGBR4,0.142848,30.58


In [54]:
list_pesos = [(1+x)/(carteira_limpa['MM'].sum()+len(carteira_limpa['MM'].values)) for x in carteira_limpa['MM'].values]
list_pesos.reverse()

list_pesos

[0.06871792523827507,
 0.06806186067512997,
 0.06797645768068676,
 0.06598012386994793,
 0.06481428513838036,
 0.0638331732744605,
 0.06373449267205353,
 0.06324973643533756,
 0.06168101181099581,
 0.06166059357454983,
 0.0613040606872257,
 0.06069852473266857,
 0.05771223076665668,
 0.05763992501320058,
 0.05704011326705888,
 0.05589548516337232]

In [49]:
carteira_limpa['Pesos'] = list_pesos
carteira_limpa

,MM,Preços,Pesos
ELET3,0.002284,36.53,0.068718
MRFG3,0.022809,20.17,0.068062
JBSS3,0.033564,35.42,0.067976
PCAR3,0.034861,24.82,0.065980
PSSA3,0.088409,20.86,0.064814
USIM5,0.099267,13.93,0.063833
GOAU4,0.105661,12.27,0.063734
BRKM5,0.106027,45.52,0.063250
EGIE3,0.134156,43.48,0.061681
GGBR4,0.142848,30.58,0.061661


In [50]:
patrimonio_inicial = 10000

carteira_limpa['Lote'] = round((carteira_limpa['Pesos'] * patrimonio_inicial/carteira_limpa['Preços'])/100,2)*100
carteira_limpa

,MM,Preços,Pesos,Lote
ELET3,0.002284,36.53,0.068718,19.0
MRFG3,0.022809,20.17,0.068062,34.0
JBSS3,0.033564,35.42,0.067976,19.0
PCAR3,0.034861,24.82,0.065980,27.0
PSSA3,0.088409,20.86,0.064814,31.0
USIM5,0.099267,13.93,0.063833,46.0
GOAU4,0.105661,12.27,0.063734,52.0
BRKM5,0.106027,45.52,0.063250,14.0
EGIE3,0.134156,43.48,0.061681,14.0
GGBR4,0.142848,30.58,0.061661,20.0


In [51]:
carteira_final = carteira_limpa[['Lote','Preços']]
carteira_final

,Lote,Preços
ELET3,19.0,36.53
MRFG3,34.0,20.17
JBSS3,19.0,35.42
PCAR3,27.0,24.82
PSSA3,31.0,20.86
USIM5,46.0,13.93
GOAU4,52.0,12.27
BRKM5,14.0,45.52
EGIE3,14.0,43.48
GGBR4,20.0,30.58
